# **Depedencies**

In [1]:
import torch
import os
from tqdm import tqdm

from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable, grad
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, utils

from math import sqrt

from datetime import datetime
import os

# **Project Parameters**

In [2]:
DATAROOT = "data"
lr = 0.001
input_code_size = 128
NUM_CHANNELS = 128
BATCH_SIZE = 4 #Batch Size during 1 iteration
START_SIZE = 1 #At which size do we start the programm
NUM_ITER = 250000 #How many iterations in total

# **Data Preparation**

In [3]:
# Create the dataset

# This time we need to create responsive dataloader that will adapt itself depending on
# the size that we are currently treating

def pokemon_loader_folder(path):
    def loader(transform, image_size):
        data = datasets.ImageFolder(path, transform=transform)
        if image_size <= 64:
            data_loader = DataLoader(data, shuffle=True, batch_size=BATCH_SIZE,
                                     num_workers=4)
        else:
            data_loader = DataLoader(data, shuffle=True, batch_size=BATCH_SIZE//2,
                         num_workers=4)
        return data_loader
    return loader

def create_loader_depending_on_image_size(dataloader, image_size):
    transform = transforms.Compose([
        transforms.Resize(image_size+int(image_size*0.2)+1),
        transforms.RandomCrop(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    loader = dataloader(transform, image_size)
    return loader

In [4]:
def upscale(feat):
    return F.interpolate(feat, scale_factor=2, mode='bilinear', align_corners=False)

# **Creating the Equalized Layers**

In [5]:
class EqualLR:
    def __init__(self, name):
        self.name = name

    def compute_weight(self, module):
        weight = getattr(module, self.name + '_orig')
        fan_in = weight.data.size(1) * weight.data[0][0].numel()

        return weight * sqrt(2 / fan_in)

    @staticmethod
    def apply(module, name):
        fn = EqualLR(name)

        weight = getattr(module, name)
        del module._parameters[name]
        module.register_parameter(name + '_orig', nn.Parameter(weight.data))
        module.register_forward_pre_hook(fn)

        return fn

    def __call__(self, module, input):
        weight = self.compute_weight(module)
        setattr(module, self.name, weight)


def equal_lr(module, name='weight'):
    EqualLR.apply(module, name)

    return module


class EqualConv2d(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

        conv = nn.Conv2d(*args, **kwargs)
        conv.weight.data.normal_()
        conv.bias.data.zero_()
        self.conv = equal_lr(conv)

    def forward(self, input):
        return self.conv(input)


class EqualConvTranspose2d(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

        conv = nn.ConvTranspose2d(*args, **kwargs)
        conv.weight.data.normal_()
        conv.bias.data.zero_()
        self.conv = equal_lr(conv)

    def forward(self, input):
        return self.conv(input)

class EqualLinear(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()

        linear = nn.Linear(in_dim, out_dim)
        linear.weight.data.normal_()
        linear.bias.data.zero_()

        self.linear = equal_lr(linear)

    def forward(self, input):
        return self.linear(input)

# **Creating the Convolution Block**

In [6]:
class ConvolutionBlock(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, padding, kernel_size2=None, padding2=None):
        super().__init__()

        pad1 = padding
        pad2 = padding
        if padding2 is not None:
            pad2 = padding2

        kernel1 = kernel_size
        kernel2 = kernel_size
        if kernel_size2 is not None:
            kernel2 = kernel_size2

        convs = [EqualConv2d(in_channel, out_channel, kernel1, padding=pad1)]
        convs.append(nn.LeakyReLU(0.1))
        convs.append(EqualConv2d(out_channel, out_channel, kernel2, padding=pad2))
        convs.append(nn.LeakyReLU(0.1))

        self.conv = nn.Sequential(*convs)

    def forward(self, input):
        out = self.conv(input)
        return out

# **Generator**

In [7]:
class Generator(nn.Module):
    def __init__(self, input_code_dim=128, in_channel=128):
        super().__init__()
        self.input_dim = input_code_dim
        self.input_layer = nn.Sequential(
            EqualConvTranspose2d(input_code_dim, in_channel, 4, 1, 0),
            nn.LeakyReLU(0.1))

        self.progression_4 = ConvolutionBlock(in_channel, in_channel, 3, 1)
        self.progression_8 = ConvolutionBlock(in_channel, in_channel, 3, 1)
        self.progression_16 = ConvolutionBlock(in_channel, in_channel, 3, 1)
        self.progression_32 = ConvolutionBlock(in_channel, in_channel, 3, 1)
        self.progression_64 = ConvolutionBlock(in_channel, in_channel//2, 3, 1)
        self.progression_128 = ConvolutionBlock(in_channel//2, in_channel//4, 3, 1)

        self.to_rgb_8 = EqualConv2d(in_channel, 3, 1)
        self.to_rgb_16 = EqualConv2d(in_channel, 3, 1)
        self.to_rgb_32 = EqualConv2d(in_channel, 3, 1)
        self.to_rgb_64 = EqualConv2d(in_channel//2, 3, 1)
        self.to_rgb_128 = EqualConv2d(in_channel//4, 3, 1)
        
        self.max_step = 5
        
    # This allows the generator to multiply the output of the previous layer, in order
    # To make it a x2 sized picture
    def progress(self, feat, module):
        out = F.interpolate(feat, scale_factor=2, mode='bilinear', align_corners=False)
        out = module(out)
        return out

    def output(self, feat1, feat2, module1, module2, alpha):
        # Depending on the alpha, we add progressively a new layer 
        if 0 <= alpha < 1:
            skip_rgb = upscale(module1(feat1))
            out = (1-alpha)*skip_rgb + alpha*module2(feat2)
        else:
            out = module2(feat2)
        return out

    def forward(self, input, step=0, alpha=-1):
        if step > self.max_step:
            step = self.max_step

        out_4 = self.input_layer(input.view(-1, self.input_dim, 1, 1))
        out_4 = self.progression_4(out_4)
        out_8 = self.progress(out_4, self.progression_8)
        if step==1:
            return self.to_rgb_8(out_8)
        
        out_16 = self.progress(out_8, self.progression_16)
        if step==2:
            return self.output( out_8, out_16, self.to_rgb_8, self.to_rgb_16, alpha )
        
        out_32 = self.progress(out_16, self.progression_32)
        if step==3:
            return self.output( out_16, out_32, self.to_rgb_16, self.to_rgb_32, alpha )

        out_64 = self.progress(out_32, self.progression_64)
        if step==4:
            return self.output( out_32, out_64, self.to_rgb_32, self.to_rgb_64, alpha )
        
        out_128 = self.progress(out_64, self.progression_128)
        if step==5:
            return self.output( out_64, out_128, self.to_rgb_64, self.to_rgb_128, alpha )

# **Discriminator**

In [8]:
class Discriminator(nn.Module):
    def __init__(self, feat_dim=128):
        super().__init__()

        self.progression = nn.ModuleList([ConvolutionBlock(feat_dim//4, feat_dim//4, 3, 1),
                                          ConvolutionBlock(feat_dim//4, feat_dim//2, 3, 1),
                                          ConvolutionBlock(feat_dim//2, feat_dim, 3, 1),
                                          ConvolutionBlock(feat_dim, feat_dim, 3, 1),
                                          ConvolutionBlock(feat_dim, feat_dim, 3, 1),
                                          ConvolutionBlock(feat_dim, feat_dim, 3, 1),
                                          ConvolutionBlock(feat_dim+1, feat_dim, 3, 1, 4, 0)])

        self.from_rgb = nn.ModuleList([EqualConv2d(3, feat_dim//4, 1),
                                       EqualConv2d(3, feat_dim//4, 1),
                                       EqualConv2d(3, feat_dim//2, 1),
                                       EqualConv2d(3, feat_dim, 1),
                                       EqualConv2d(3, feat_dim, 1),
                                       EqualConv2d(3, feat_dim, 1),
                                       EqualConv2d(3, feat_dim, 1)])

        self.n_layer = len(self.progression)

        self.linear = EqualLinear(feat_dim, 1)

    def forward(self, input, step=0, alpha=-1):
        for i in range(step, -1, -1):
            index = self.n_layer - i - 1

            if i == step:
                out = self.from_rgb[index](input)

            if i == 0:
                out_std = torch.sqrt(out.var(0, unbiased=False) + 1e-8)
                mean_std = out_std.mean()
                mean_std = mean_std.expand(out.size(0), 1, 4, 4)
                out = torch.cat([out, mean_std], 1)

            out = self.progression[index](out)

            if i > 0:
                out = F.interpolate(out, scale_factor=0.5, mode='bilinear', align_corners=False)

                if i == step and 0 <= alpha < 1:
                    skip_rgb = F.interpolate(input, scale_factor=0.5, mode='bilinear', align_corners=False)
                    skip_rgb = self.from_rgb[index + 1](skip_rgb)
                    out = (1 - alpha) * skip_rgb + alpha * out

        out = out.squeeze(2).squeeze(2)
        out = self.linear(out)

        return out

# Training Function

In [9]:
def train(generator, discriminator, start_size, loader, num_iter=50000):
    step = start_size # can be 1 = 8, 2 = 16, 3 = 32, 4 = 64, 5 = 128
    data_loader = create_loader_depending_on_image_size(loader, 4 * 2 ** step)
    dataset = iter(data_loader)

    num_iter_remain = num_iter - (num_iter//5)*(step-1)

    # Progress Bar 
    pbar = tqdm(range(num_iter_remain))

    disc_loss_val = 0
    gen_loss_val = 0
    grad_loss_val = 0

    # Creating Saving Folder
    log_folder = 'log_folder'
    
    os.mkdir(log_folder)
    # To store the pictures
    os.mkdir(log_folder+'/sample')

    # Log for Discriminator & Generator Loss Value
    log_file_name = os.path.join(log_folder, 'generator_discriminator_values')
    log_file = open(log_file_name, 'w')
    log_file.write('generator,discriminator,gradient,alpha\n')
    log_file.close()

    # Initializing the parameters
    cuda0=torch.device('cuda:0')
    alpha = 0
    one = torch.tensor(1, dtype=torch.float, device=cuda0)
    mone = one * -1
    iteration = 0

    for i in pbar:
        # Clears up old gradients
        discriminator.zero_grad()

        # Alpha is the evolution rate, that will progressively
        # add the new output produced by the new layer added on top of the other
        # The more we enter in the iteration 
        alpha = min(1, (2/(num_iter//5)) * iteration)

        # If the number of iteration goes over the number 1/5 of the total iteration
        # We go to the next step and increase the size of the produced image.
        if iteration > num_iter//5:
            alpha = 0
            iteration = 0
            step += 1
            data_loader = create_loader_depending_on_image_size(loader, 4 * 2 ** step)
            dataset = iter(data_loader)

        try:
            real_image, label = next(dataset)
        except (OSError, StopIteration):
            # When the dataset iterator is over, we need to reload it
            dataset = iter(data_loader)
            real_image, label = next(dataset)

        iteration += 1

        ### 1. train Discriminator
        ## Takes a new image and compare the prediction vs 
        b_size = real_image.size(0)
        real_image = real_image.to(device)
        label = label.to(device)
        real_predict = discriminator(real_image, step=step, alpha=alpha)
        real_predict = real_predict.mean() - 0.001 * (real_predict ** 2).mean()
        real_predict.backward(mone)

        # sample input data: vector for Generator
        # We generate a fake vector
        gen_z = torch.randn(b_size, input_code_size).to(device)

        fake_image = generator(gen_z, step=step, alpha=alpha)
        fake_predict = discriminator(fake_image.detach(), step=step, alpha=alpha)
        fake_predict = fake_predict.mean()
        fake_predict.backward(one)


        eps = torch.rand(b_size, 1, 1, 1).to(device)
        x_hat = eps * real_image.data + (1 - eps) * fake_image.detach().data
        x_hat.requires_grad = True
        hat_predict = discriminator(x_hat, step=step, alpha=alpha)
        grad_x_hat = grad(outputs=hat_predict.sum(), inputs=x_hat, create_graph=True)[0]
        grad_penalty = ((grad_x_hat.view(grad_x_hat.size(0), -1).norm(2, dim=1) - 1)**2).mean()
        grad_penalty = 10 * grad_penalty
        grad_penalty.backward()
        grad_loss_val += grad_penalty.item()
        disc_loss_val += (real_predict - fake_predict).item()

        # Take a step based on the gradient, for the optimizer
        d_optimizer.step()

        ### 2. train Generator
        generator.zero_grad()
        discriminator.zero_grad()
        
        predict = discriminator(fake_image, step=step, alpha=alpha)

        loss = -predict.mean()
        gen_loss_val += loss.item()


        loss.backward()
        g_optimizer.step()
        accumulate(g_running, generator)

        # Save the images into a folder
        if (i + 1) % 1000 == 0 or i==0:
            with torch.no_grad():
                images = g_running(torch.randn(5 * 10, input_code_size).to(device), step=step, alpha=alpha).data.cpu()

                utils.save_image(
                    images,
                    f'{log_folder}/sample/{str(i + 1).zfill(6)}.png',
                    nrow=10,
                    normalize=True,
                    range=(-1, 1))
        
        # Save the Generator Loss Value, Discriminator Loss Value and the Alpha value every 500 iterations
        if (i+1)%500 == 0:
            state_msg = (f'{i + 1}; G: {gen_loss_val/500:.3f}; D: {disc_loss_val/500:.3f};'
                f' Grad: {grad_loss_val/500:.3f}; Alpha: {alpha:.3f}')
            
            log_file = open(log_file_name, 'a+')
            new_line = "%.5f,%.5f,%.5f,%.5f\n"%(gen_loss_val/500, disc_loss_val/500,grad_loss_val/500, alpha)
            log_file.write(new_line)
            log_file.close()

            disc_loss_val = 0
            gen_loss_val = 0
            grad_loss_val = 0

            print(state_msg)

# **Running the Model**

In [10]:
def accumulate(model1, model2, decay=0.999):
    par1 = dict(model1.named_parameters())
    par2 = dict(model2.named_parameters())

    for k in par1.keys():
        par1[k].data.mul_(decay).add_(1 - decay, par2[k].data)

In [ ]:
device = torch.device("cuda:0")

generator = Generator(in_channel=NUM_CHANNELS, input_code_dim=input_code_size).to(device)
discriminator = Discriminator(feat_dim=NUM_CHANNELS).to(device)
g_running = Generator(in_channel=NUM_CHANNELS, input_code_dim=input_code_size).to(device)

g_running.train(False)

g_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.0, 0.99))
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.0, 0.99))

accumulate(g_running, generator, 0)

loader = pokemon_loader_folder(DATAROOT)

train(generator, discriminator, START_SIZE, loader, NUM_ITER)

  0%|          | 504/250000 [00:15<2:19:10, 29.88it/s]

500; G: -0.054; D: 0.056; Grad: 0.069; Alpha: 0.020


  0%|          | 1006/250000 [00:32<2:07:15, 32.61it/s]

1000; G: 0.356; D: -0.017; Grad: 0.062; Alpha: 0.040


  1%|          | 1504/250000 [00:47<2:32:10, 27.22it/s]

...

 12%|█▏        | 29504/250000 [14:55<1:45:41, 34.77it/s]

29500; G: 0.256; D: 0.086; Grad: 0.053; Alpha: 1.000


 12%|█▏        | 30005/250000 [15:11<1:53:31, 32.30it/s]

30000; G: 0.227; D: 0.067; Grad: 0.051; Alpha: 1.000


 12%|█▏        | 30059/250000 [15:13<2:09:50, 28.23it/s]

# **Générer ses propres Pokemons**

In [ ]:
images = g_running(torch.randn(5 * 10, input_code_size).to(device), step=step, alpha=alpha).data.cpu()
images

# **Etude des Loss et Gradient**

In [ ]:
import os
file_path = os.path.join(os.getcwd(), 'log_folder/generator_discriminator_values')

%matplotlib inline
import matplotlib.pyplot as plt

with open(file_path, 'r') as file:
    logs_dict = {}
    titles = next(file).rstrip('\n').split(',')
    for title in titles:
        logs_dict[title] = []
    file_reader = file.readlines()
    for line in file_reader:
        values = list(map(float, line.rstrip('\n').split(',')))
        for i in range(len(titles)):
            logs_dict[titles[i]].append(values[i])

X = list(range(500, 53500, 500))
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(X, logs_dict['generator'],label="G")
plt.plot(X, logs_dict['discriminator'],label="D")
plt.plot(X, logs_dict['alpha'],label="A")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Gradient values during training")
plt.plot(X, logs_dict['gradient loss'],label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()